In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import serial
import time
from pushbullet import Pushbullet
import easyocr

# EasyOCR 리더 초기화 (한국어 및 영어 지원)
reader = easyocr.Reader(['ko', 'en'])

try:
    arduino = serial.Serial('COM5', 9600, timeout=1)
    time.sleep(2)
    print("아두이노 연결 완료")
except serial.SerialException as e:
    print("아두이노 연결 실패:", e)
    arduino = None

det_model = YOLO("yolov8n.pt") 
class_names = det_model.names

cam = cv2.VideoCapture(0)
if not cam.isOpened():
    print(" 카메라 열기 실패")
    exit()
print(" 카메라 연결 완료")

last_send_time = time.time()

while True:
    ret, frame = cam.read()
    if not ret:
        print(" 프레임 읽기 실패")
        break

    scale = 1280 / max(frame.shape)
    if scale < 1:
        frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

    display_frame = frame.copy()

    frame_height, frame_width = frame.shape[:2]
    roi_height = int(frame_height * 0.3)
    roi_y = (frame_height - roi_height) // 2
    roi = (0, roi_y, frame_width, roi_height)

    cv2.rectangle(display_frame, (roi[0], roi[1]), (roi[0]+roi[2], roi[1]+roi[3]), (255, 0, 0), 2)

    results = det_model(frame, verbose=False)
    boxes = results[0].boxes

    car_detected = False
    plate_text = ""

    for box in boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        label = class_names.get(cls_id, "unknown")
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        if label.lower() == "car" and conf > 0.5:
            if x1 < roi[0]+roi[2] and x2 > roi[0] and y1 < roi[1]+roi[3] and y2 > roi[1]:
                car_detected = True
                car_crop = frame[y1:y2, x1:x2]

                # 번호판 인식 시도
                results_ocr = reader.readtext(car_crop)
                if results_ocr:
                    plate_text = results_ocr[0][1]
                    print("번호판 인식:", plate_text)

                # 화면에 박스 및 텍스트 표시
                cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(display_frame, f"{label} {conf:.2f}", (x1, max(y1 - 20, 0)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                if plate_text:
                    cv2.putText(display_frame, f"Plate: {plate_text}", (x1, max(y1 - 5, 0)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    current_time = time.time()
    if current_time - last_send_time >= 1.25 and arduino is not None:
        if car_detected:
            arduino.write(b'C')
            print(" 아두이노 전송: 차량 감지")
            pb = Pushbullet("o.0dwbnbFUsUaXRdTk9cFCt3dx1R5O2All")
            push = pb.push_note("경고", "건널목에 차량이 있습니다.")
            print("message sent!")
        else:
            arduino.write(b'N')
            print("📤 아두이노 전송: 차량 없음")
        last_send_time = current_time

    cv2.imshow('cam', display_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

if arduino is not None:
    arduino.close()
cam.release()
cv2.destroyAllWindows()
